In [2]:
# Importamos todas las librerías necesarias para utilizar Spark Streaming con Python desde
# JUpyter Notebook

#import findspark
#findspark.init()
from datetime import datetime
import time
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
!pip3 install numpy
import numpy
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from pyspark.ml import Pipeline

In [3]:
# Iniciamos la sesión de Spark indicando el master empleado y la configuración utilizada
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.2")\
    .getOrCreate()

#spark = sparknlp.start()

2022-03-30 18:59:52,713 WARN util.Utils: Your hostname, fabio resolves to a loopback address: 127.0.1.1; using 192.168.100.23 instead (on interface eno1)
2022-03-30 18:59:52,714 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/spark-3.2.1-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/fabio/.ivy2/cache
The jars for the packages stored in: /home/fabio/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9d97203b-dd2d-41fb-acad-c6da7fe8245c;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.4.2 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok

In [4]:
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, Params, TypeConverters
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
from SentiLib.text import predict_emotions_text
from pyspark.sql.functions import udf

class CustomTransformer(Transformer, HasInputCol, HasOutputCol, DefaultParamsReadable, DefaultParamsWritable):
    input_col = Param(Params._dummy(), "input_col", "input column name.", typeConverter=TypeConverters.toString)
    output_col = Param(Params._dummy(), "output_col", "output column name.", typeConverter=TypeConverters.toString)
  
    @keyword_only
    def __init__(self, input_col: str = "input", output_col: str = "output"):
        super(CustomTransformer, self).__init__()
        self._setDefault(input_col=None, output_col=None)
        kwargs = self._input_kwargs
        self.set_params(**kwargs)
    
    @keyword_only
    def set_params(self, input_col: str = "input", output_col: str = "output"):
        kwargs = self._input_kwargs
        self._set(**kwargs)
    
    def get_input_col(self):
        return self.getOrDefault(self.input_col)
  
    def get_output_col(self):
        return self.getOrDefault(self.output_col)
  
    def _transform(self, df: DataFrame):
        input_col = self.get_input_col()
        output_col = self.get_output_col()
        # The custom action: concatenate the integer form of the doubles from the Vector
        transform_udf = F.udf(lambda x: str(predict_emotions_text(x)), StringType())
        return df.withColumn(output_col, transform_udf(input_col))

2022-03-30 19:00:07.468197: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib
2022-03-30 19:00:07.468216: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
# Tomamos de la carpeta "Datos" los archivos txt que van entrando con las diferentes frases
# Estos archivos se toman de uno en uno.
text = spark.readStream \
.format("text") \
.text("file:///home/user/Datos")
print(sparknlp.version())
print(spark.version)
Document = DocumentAssembler().setInputCol('value').setOutputCol('document').setCleanupMode('shrink')
Sentence = SentenceDetector().setInputCols('document').setOutputCol('sentence')
Sentence.setExplodeSentences(True)
Tokenizer = RegexTokenizer().setInputCols('sentence').setOutputCol('token')
Normalizer = Normalizer().setInputCols(['token']).setOutputCol('normalize').setLowercase(True).setCleanupPatterns([])
Stemmer = Stemmer().setInputCols('normalize').setOutputCol('stemming')
TokenAssembler = TokenAssembler().setInputCols("sentence", "stemming").setOutputCol("cleanText")
Finisher = Finisher().setInputCols('cleanText').setOutputCols('output').setOutputAsArray(False)
custom_transformer = CustomTransformer(input_col="output", output_col="sentiment")

pipeline = Pipeline().setStages([
    Document,
    Sentence,
    Tokenizer,
    Normalizer,
    Stemmer,
    TokenAssembler,
    Finisher,
    custom_transformer
])

model = pipeline.fit(text)
result = model.transform(text)

final = result.withColumn("timestamp", current_timestamp())

3.4.1
3.2.1


In [ ]:
print(datetime.now())

query =final.select('sentiment', 'timestamp') \
.writeStream \
.outputMode("append") \
.format("console") \
.option("maxFilesPerTrigger", 1) \
.trigger(once=True) \
.start().awaitTermination()

print(datetime.now())


2022-03-30 19:24:31.880612


2022-03-30 19:24:31,900 WARN streaming.ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3e52494f-b9bc-49c8-883f-5084451442b6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
2022-03-30 19:24:31,908 WARN streaming.ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
2022-03-30 19:24:36.342157: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib
2022-03-30 19:24:36.342179: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
